作业一：用GaussianNB计算鸢尾花。。。。

In [7]:
import numpy as np
import pandas as pd

In [8]:
from sklearn.datasets import load_iris
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    return data[:,:2], data[:,-1]

In [11]:
X, Y = create_data()

In [12]:
X.shape

(100, 2)

In [14]:
Y.shape

(100,)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state=1)  #random_state随机数种子

In [82]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train,Y_train)
GaussianNB(priors=None)

GaussianNB(priors=None)

In [27]:
clf.score(X_test,Y_test)

1.0

In [44]:
clf_pf = GaussianNB()
clf_pf.partial_fit(X, Y, np.unique(Y))
GaussianNB(priors=None)

GaussianNB(priors=None)

In [45]:
clf.score(X_test,Y_test)

1.0

作业二：用GaussianNB进行文本分类

In [75]:
def loadDataSet():
    postingList =[['my','dog','has','flea','problems','help','please'],
                  ['maybe','not','take','him','to','dog','park','stupid'],
                  ['my','dalmation','is','so','cute','I','love','him'],
                  ['stop','posting','stupid','worthless','grabage'],
                  ['mr','licks','ate','my','steak','how','to','stop','him'],
                  ['quit','buying','worthless','dog','food','stupid']]
    classVec = np.array([0,1,0,1,0,1]) 
    
    return postingList,classVec

In [76]:
postingList,classVec = loadDataSet()

In [83]:
def create_Vocabulary(postingList):
    Voc_list = set([])
    for words in postingList:
        Voc_list = Voc_list | set(words)      #并集|
    return list(Voc_list)

In [85]:
Vocabulary = create_Vocabulary(postingList)

In [86]:
def words2vec(Vocabulary,postingList):        #词转向量
    m,n = len(postingList),len(Vocabulary)
    Words_Vec = np.zeros((m,n))
    for i in range(m):
        for word in postingList[i]:
            try:
                index = Vocabulary.index(word)
                Words_Vec[i,index] = 1
            except:
                print('这个{}不在这个词集'.format(word))
               # exit(0)
    return Words_Vec

In [88]:
Words_Vec = words2vec(Vocabulary,postingList)
Words_Vec.shape

(6, 32)

In [89]:
classVec.shape

(6,)

In [111]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Words_Vec,classVec,test_size = 0.2,random_state=1)  #random_state随机数种子

In [112]:
from sklearn.naive_bayes import GaussianNB
clf_pf = GaussianNB()
clf_pf.partial_fit(Words_Vec, classVec, np.unique(classVec))
GaussianNB(priors=None)

GaussianNB(priors=None)

In [113]:
clf.score(X_test,Y_test)

1.0

In [114]:
clf = GaussianNB()
clf.fit(X_train,Y_train)
GaussianNB(priors=None)

GaussianNB(priors=None)

In [115]:
clf.score(X_test,Y_test)

1.0

作业三：将先验概率泛化

In [1]:
def Model(X,y,Vocabulary,X_test,gamma):
    unique_y = np.unique(y)
    K = len(unique_y)
    predict_=[]
    y_fenmu = classVec.size+(K*gamma)
    p_y={}
    for i in classVec:
        if i not in p_y:
            p_y[i] = 1
        else:
            p_y[i] += 1
    for key,value in p_y.items():
        p_y[key] = (value + gamma)/ y_fenmu
    test_x = words2vec(Vocabulary,X_test)
    for label in unique_y:
        P_y = p_y[label]
        Index = np.where(y == label)[0]
        split_x = X[Index]           #为0，1的标签Y对应的特征X
        Sj = 2
        cond_fenmu = split_x.shape[0] + (Sj * gamma)
        sum_feature = np.sum(test_x == split_x,axis=0) + gamma   #列之间相加
        cond_prob = np.product(sum_feature/cond_fenmu)
        houyan = P_y * cond_prob
        predict_.append([label,houyan])
    predict = sorted(predict_,key=lambda z:z[1],reverse=True)[0][0]
    return predict